In [33]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
pd.set_option('display.max_columns', 1000)

In [35]:
# Defining directories
dir_path = Path.cwd().parent
data_folder = dir_path.joinpath("data/processed")

#training frames
s1_path = data_folder.joinpath("sent_1_season_medians.parquet")
s2_path = data_folder.joinpath("sent_2_season_medians.parquet")
# cov_path = data_folder.joinpath("cov_final.parquet")

species_dict_path = dir_path.joinpath("data/species_dict.json")

index_cols = ["long", "lat", 'year', 'season', "species_names"]


# previous model frame path 
actual_model_path = dir_path.joinpath("data/processed_532 features/species_train_df.csv")

out_folder = dir_path.joinpath("docs/boxplots/actual_values")

In [36]:
s1 = pd.read_parquet(s1_path).drop('month', axis=1)
s1

,long,lat,year,season,species_names,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending,VV_VH_A,VV_VH_D,VH_VV_A,VH_VV_D,SAR_NDVI_A,SAR_NDVI_D,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
0,75.66669580000007,32.25107380000003,2018,summer,Bambusoideae spp.,22.552725,19.700091,23.384565,19.956352,1.179084,1.156924,0.848424,0.865649,-0.082086,-0.072357,-3.582735,-2.830202,43.769740,41.528996,-0.330660,-0.291276,-0.163029,-0.143803,-156.466199,-111.066850
1,75.66669580000007,32.25107380000003,2018,winter,Bambusoideae spp.,23.639657,20.492180,22.741578,20.045554,1.130519,1.120752,0.884551,0.892258,-0.061261,-0.056938,-2.566007,-2.474472,44.383245,44.359597,-0.245967,-0.228494,-0.122064,-0.113509,-107.481875,-107.537649
2,75.66669580000007,32.25107380000003,2019,summer,Bambusoideae spp.,23.856063,19.956352,21.846914,20.530784,1.064105,1.198059,0.939757,0.834683,-0.031057,-0.090106,-1.316130,-3.943802,42.433853,43.768323,-0.124348,-0.363376,-0.062054,-0.178761,-55.774555,-172.613601
3,75.66669580000007,32.25107380000003,2019,winter,Bambusoideae spp.,23.135292,20.770598,24.087430,20.293079,1.168957,1.113785,0.855464,0.897885,-0.077898,-0.053817,-3.309932,-2.364695,44.380509,43.905890,-0.313493,-0.215900,-0.154856,-0.107319,-140.641534,-103.918188
4,75.66669580000007,32.25107380000003,2020,summer,Bambusoideae spp.,23.268281,21.227070,22.068259,20.086002,1.104880,1.196235,0.905535,0.836198,-0.049694,-0.089273,-2.090337,-4.028260,42.235219,43.871136,-0.199344,-0.360037,-0.099106,-0.177096,-87.990981,-171.256533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084693,78.12624330000006,31.258713300000068,2022,winter,Rhododendron arboreum,20.606978,17.320957,22.648178,21.522883,1.063837,1.147706,0.940453,0.871303,-0.030805,-0.068774,-1.395479,-2.559417,44.171062,37.472564,-0.123384,-0.276403,-0.061530,-0.136900,-63.214959,-97.142729
2084694,78.12624330000006,31.258713300000068,2023,summer,Rhododendron arboreum,18.631598,16.617292,23.481170,19.839611,1.182774,1.045365,0.846055,0.956604,-0.083549,-0.022179,-3.641559,-0.741017,43.701799,35.134575,-0.336719,-0.088761,-0.165855,-0.044337,-158.956064,-28.777109
2084695,78.12624330000006,31.258713300000068,2023,winter,Rhododendron arboreum,20.718820,16.989700,22.944662,20.334238,1.140720,1.103533,0.876639,0.906180,-0.065735,-0.049219,-2.808266,-1.894584,43.711418,37.708520,-0.264081,-0.197352,-0.130904,-0.098199,-119.971852,-64.619521
2084696,78.12624330000006,31.258713300000068,2024,summer,Rhododendron arboreum,19.612070,18.042630,24.099331,20.606978,1.125739,1.085927,0.888306,0.922582,-0.059151,-0.040712,-2.713898,-1.569440,44.904501,37.654700,-0.237433,-0.163345,-0.117889,-0.081177,-124.516749,-60.586361


In [37]:
s2 = pd.read_parquet(s2_path).drop('month', axis=1)

In [38]:
merged_df = reduce(lambda left, right: pd.merge(left, right, on=index_cols, how='outer', validate="1:1"), [s1, s2])
merged_df



,long,lat,year,season,species_names,vv_Descending,vh_Descending,vv_Ascending,vh_Ascending,VV_VH_A,VV_VH_D,VH_VV_A,VH_VV_D,SAR_NDVI_A,SAR_NDVI_D,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,75.66669580000007,32.25107380000003,2018,summer,Bambusoideae spp.,22.552725,19.700091,23.384565,19.956352,1.179084,1.156924,0.848424,0.865649,-0.082086,-0.072357,-3.582735,-2.830202,43.769740,41.528996,-0.330660,-0.291276,-0.163029,-0.143803,-156.466199,-111.066850,0.28970,0.15440,0.23560,0.32420,0.17640,0.17720,0.000000,-0.180000,0.377670,1.910389,0.707107,0.204080,0.262364,0.197548,0.159463,0.173049,0.251835,-0.776231,13.339193,0.794514,36.628039,0.185947,0.287273
1,75.66669580000007,32.25107380000003,2018,winter,Bambusoideae spp.,23.639657,20.492180,22.741578,20.045554,1.130519,1.120752,0.884551,0.892258,-0.061261,-0.056938,-2.566007,-2.474472,44.383245,44.359597,-0.245967,-0.228494,-0.122064,-0.113509,-107.481875,-107.537649,0.24600,0.13630,0.18410,0.34990,0.14930,0.15820,-0.027352,-0.212001,0.439325,2.151097,0.687494,0.300179,0.387179,0.289427,0.258520,0.324125,0.361074,-0.750811,21.943751,1.078607,18.783879,0.240258,0.411741
2,75.66669580000007,32.25107380000003,2019,summer,Bambusoideae spp.,23.856063,19.956352,21.846914,20.530784,1.064105,1.198059,0.939757,0.834683,-0.031057,-0.090106,-1.316130,-3.943802,42.433853,43.768323,-0.124348,-0.363376,-0.062054,-0.178761,-55.774555,-172.613601,0.25885,0.11280,0.19540,0.29230,0.13280,0.12965,0.021650,-0.154670,0.477402,2.683532,0.722090,0.249000,0.295781,0.237530,0.196948,0.200573,0.325497,-0.807782,18.051811,0.916986,-36.203756,0.192434,0.400403
3,75.66669580000007,32.25107380000003,2019,winter,Bambusoideae spp.,23.135292,20.770598,24.087430,20.293079,1.168957,1.113785,0.855464,0.897885,-0.077898,-0.053817,-3.309932,-2.364695,44.380509,43.905890,-0.313493,-0.215900,-0.154856,-0.107319,-140.641534,-103.918188,0.25620,0.14105,0.18470,0.38535,0.15455,0.16150,-0.021569,-0.205235,0.465031,2.293912,0.691602,0.328547,0.406719,0.317695,0.283551,0.352000,0.382530,-0.715748,34.403773,1.136096,50.859088,0.269128,0.427497
4,75.66669580000007,32.25107380000003,2020,summer,Bambusoideae spp.,23.268281,21.227070,22.068259,20.086002,1.104880,1.196235,0.905535,0.836198,-0.049694,-0.089273,-2.090337,-4.028260,42.235219,43.871136,-0.199344,-0.360037,-0.099106,-0.177096,-87.990981,-171.256533,0.27645,0.14465,0.21670,0.34525,0.16110,0.16560,-0.012700,-0.194859,0.407303,2.025796,0.697924,0.264676,0.305009,0.255691,0.223837,0.226889,0.318257,-0.755411,170.718932,0.911091,140.143094,0.224103,0.360856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084693,78.12624330000006,31.258713300000068,2022,winter,Rhododendron arboreum,20.606978,17.320957,22.648178,21.522883,1.063837,1.147706,0.940453,0.871303,-0.030805,-0.068774,-1.395479,-2.559417,44.171062,37.472564,-0.123384,-0.276403,-0.061530,-0.136900,-63.214959,-97.142729,0.16190,0.13255,0.12955,0.17620,0.11455,0.12110,-0.021048,-0.204626,0.168492,1.384757,0.692029,0.106228,0.297474,0.098332,0.088290,0.152568,0.158703,-0.922615,4.950380,0.721901,5.386990,0.090058,0.212149
2084694,78.12624330000006,31.258713300000068,2023,summer,Rhododendron arboreum,18.631598,16.617292,23.481170,19.839611,1.182774,1.045365,0.846055,0.956604,-0.083549,-0.022179,-3.641559,-0.741017,43.701799,35.134575,-0.336719,-0.088761,-0.165855,-0.044337,-158.956064,-28.777109,0.23905,0.13985,0.17975,0.33210,0.14780,0.16015,-0.040811,-0.227749,0.311191,1.728078,0.677616,0.273302,0.367826,0.263232,0.251601,0.297785,0.334304,-0.768417,5.257732,1.015502,17.123828,0.220954,0.384380
2084695,78.12624330000006,31.258713300000068,2023,winter,Rhododendron arboreum,20.718820,16.989700,22.944662,20.334238,1.140720,1.103

### Widening the frame

In [39]:
merged_df_wide = merged_df.pivot(index=['long', 'lat', 'year','species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in merged_df_wide.columns]
merged_df_wide.columns = new_cols
merged_df_wide

,long,lat,year,species_names,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter
0,75.66669580000007,32.25107380000003,2018,Bambusoideae spp.,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.000000,-0.027352,-0.180000,-0.212001,0.377670,0.439325,1.910389,2.151097,0.707107,0.687494,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741
1,75.66669580000007,32.25107380000003,2019,Bambusoideae spp.,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.021650,-0.021569,-0.154670,-0.205235,0.477402,0.465031,2.683532,2.293912,0.722090,0.691602,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497
2,75.66669580000007,32.25107380000003,2020,Bambusoideae spp.,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,-0.012700,-0.007424,-0.194859,-0.188686,0.407303,0.466377,2.025796,2.256043,0.697924,0.701786,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.844088,0.911091,1.057878,140.143094,33.429727,0.224103,0.243441,0.360856,0.397863
3,75.66669580000007,32.25107380000003,2021,Bambusoideae spp.,24.927604,24.248816,19.777236,20.024829,22.430380,22.405492,20.293838,20.827854,1.136516,1.130413,1.218761,1.162297,0.882609,0.884633,0.820505,0.860365,-0.0630

### Species Codes

In [40]:

with open(species_dict_path, 'r') as json_file:
    species_dict = json.load(json_file)

# Swapping keys and values
swapped_dict = {value: key for key, value in species_dict.items()}
swapped_dict

{'Aesculus indica': '0',
 'Bambusoideae spp.': '1',
 'Cedrus deodara': '2',
 'Dendrocalamus strictus': '3',
 'Eucalyptus globulus': '4',
 'Lantana camara': '5',
 'Mangifera sylvatica': '6',
 'Phyllanthus emblica': '7',
 'Pinus roxburghii': '8',
 'Quercus leucotrichophora': '9',
 'Rhododendron arboreum': '10',
 'Senegalia catechu': '11',
 'Shorea robusta': '12',
 'Taxus baccata': '13'}

In [44]:
merged_df_wide["species"] = merged_df_wide['species_names'].map(swapped_dict).astype(int)
merged_df_wide.drop('species_names', axis=1, inplace=True)
merged_df_wide

,long,lat,year,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,species
0,75.66669580000007,32.25107380000003,2018,22.552725,23.639657,19.700091,20.492180,23.384565,22.741578,19.956352,20.045554,1.179084,1.130519,1.156924,1.120752,0.848424,0.884551,0.865649,0.892258,-0.082086,-0.061261,-0.072357,-0.056938,-3.582735,-2.566007,-2.830202,-2.474472,43.769740,44.383245,41.528996,44.359597,-0.330660,-0.245967,-0.291276,-0.228494,-0.163029,-0.122064,-0.143803,-0.113509,-156.466199,-107.481875,-111.066850,-107.537649,0.289700,0.24600,0.154400,0.13630,0.235600,0.18410,0.324200,0.34990,0.17640,0.14930,0.17720,0.15820,0.000000,-0.027352,-0.180000,-0.212001,0.377670,0.439325,1.910389,2.151097,0.707107,0.687494,0.204080,0.300179,0.262364,0.387179,0.197548,0.289427,0.159463,0.258520,0.173049,0.324125,0.251835,0.361074,-0.776231,-0.750811,13.339193,21.943751,0.794514,1.078607,36.628039,18.783879,0.185947,0.240258,0.287273,0.411741,1
1,75.66669580000007,32.25107380000003,2019,23.856063,23.135292,19.956352,20.770598,21.846914,24.087430,20.530784,20.293079,1.064105,1.168957,1.198059,1.113785,0.939757,0.855464,0.834683,0.897885,-0.031057,-0.077898,-0.090106,-0.053817,-1.316130,-3.309932,-3.943802,-2.364695,42.433853,44.380509,43.768323,43.905890,-0.124348,-0.313493,-0.363376,-0.215900,-0.062054,-0.154856,-0.178761,-0.107319,-55.774555,-140.641534,-172.613601,-103.918188,0.258850,0.25620,0.112800,0.14105,0.195400,0.18470,0.292300,0.38535,0.13280,0.15455,0.12965,0.16150,0.021650,-0.021569,-0.154670,-0.205235,0.477402,0.465031,2.683532,2.293912,0.722090,0.691602,0.249000,0.328547,0.295781,0.406719,0.237530,0.317695,0.196948,0.283551,0.200573,0.352000,0.325497,0.382530,-0.807782,-0.715748,18.051811,34.403773,0.916986,1.136096,-36.203756,50.859088,0.192434,0.269128,0.400403,0.427497,1
2,75.66669580000007,32.25107380000003,2020,23.268281,23.263359,21.227070,19.558451,22.068259,22.833012,20.086002,20.043214,1.104880,1.177081,1.196235,1.196378,0.905535,0.849559,0.836198,0.835856,-0.049694,-0.081339,-0.089273,-0.089410,-2.090337,-3.590219,-4.028260,-3.818532,42.235219,42.376016,43.871136,43.480976,-0.199344,-0.327523,-0.360037,-0.360521,-0.099106,-0.161609,-0.177096,-0.177402,-87.990981,-151.061373,-171.256533,-163.082568,0.276450,0.25100,0.144650,0.13035,0.216700,0.18535,0.345250,0.35955,0.16110,0.15440,0.16560,0.15675,-0.012700,-0.007424,-0.194859,-0.188686,0.407303,0.466377,2.025796,2.256043,0.697924,0.701786,0.264676,0.296029,0.305009,0.381638,0.255691,0.285919,0.223837,0.248274,0.226889,0.318541,0.318257,0.352093,-0.755411,-0.741267,170.718932,189.844088,0.911091,1.057878,140.143094,33.429727,0.224103,0.243441,0.360856,0.397863,1
3,75.66669580000007,32.25107380000003,2021,24.927604,24.248816,19.777236,20.024829,22.430380,22.405492,20.293838,20.827854,1.136516,1.130413,1.218761,1.162297,0.882609,0.884633,0.820505,0.860365,-0.063077,-0.061215,-0.098596,-0.075058,-2.785947,-2.730013,-4.474374,-3.436265

In [26]:
merged_df_wide['species'].unique()

array([ 1,  4, 11,  6,  5,  7,  8,  3, 13,  2,  0, 10, 12,  9])

### Making Box Plots

In [28]:
west_low_species = [11, 8, 7, 6, 5, 4, 3, 1] #target
mid_line_species = [13, 12, 10, 9, 2, 0] #control

In [29]:
def sent_boxplots(data_frame, out_path, cols, plt_rows, plt_cols):
    fig, axes = plt.subplots(plt_rows, plt_cols, figsize=(30, 30))  # Create 6x7 grid
    flattened_axes = axes.flatten()

    legend_handles = None  # To store legend elements

    for i, col in enumerate(cols):
        ax = flattened_axes[i]
        sns.boxplot(data=data_frame, x="year", y=col, hue="species", orient="v", ax=ax, showfliers=False)
        ax.set_title(col)  # Label each plot

        # Remove axis labels
        ax.set_xlabel("")
        ax.set_ylabel("")

        # Store legend elements from the first plot with a legend
        if legend_handles is None:
            legend_handles, legend_labels = ax.get_legend_handles_labels()

        # Remove legends from individual subplots
        if ax.get_legend() is not None:
            ax.legend_.remove()

    # Hide unused subplots
    for j in range(i + 1, len(flattened_axes)):
        flattened_axes[j].set_visible(False)

    # Add a single legend in the bottom-right subplot
    legend_ax = flattened_axes[-1]  # Choose last subplot for legend
    legend_ax.set_visible(True)  # Ensure it's visible
    custom_labels = [species_dict.get(label, label) for label in legend_labels]
    legend_ax.legend(legend_handles, custom_labels, loc="center left", title="Species")
    legend_ax.axis("off")  # Hide axis for cleaner display

    plt.tight_layout()
    plt.savefig(out_path, dpi=300, bbox_inches="tight")  # Saves as 300 DPI PNG
    # plt.show()


In [53]:
merged_df_wide.columns[43:89]

Index(['B11summer', 'B11winter', 'B02summer', 'B02winter', 'B12summer',
       'B12winter', 'B08summer', 'B08winter', 'B04summer', 'B04winter',
       'B03summer', 'B03winter', 'NDVIsummer', 'NDVIwinter', 'ARVI2summer',
       'ARVI2winter', 'BWDRVIsummer', 'BWDRVIwinter', 'CVIsummer', 'CVIwinter',
       'CTVIsummer', 'CTVIwinter', 'EVI2summer', 'EVI2winter', 'GVMIsummer',
       'GVMIwinter', 'MSVAIhypersummer', 'MSVAIhyperwinter', 'MTVI2summer',
       'MTVI2winter', 'MNDVIsummer', 'MNDVIwinter', 'OSAVIsummer',
       'OSAVIwinter', 'PVIsummer', 'PVIwinter', 'SARVIsummer', 'SARVIwinter',
       'SLAVIsummer', 'SLAVIwinter', 'TSAVI2summer', 'TSAVI2winter',
       'WDVIsummer', 'WDVIwinter', 'WDRVIsummer', 'WDRVIwinter'],
      dtype='object')

### Intra Specie

##### Sentinel 1

In [ ]:
boxplot_out_folder = out_folder.joinpath(f"intra/sentinel_1")

if not boxplot_out_folder.exists():
    boxplot_out_folder.mkdir(parents=True)

for specie in range(0,14,1):
    species_df = merged_df_wide[merged_df_wide['species']==specie]

    boxplot_out_path = boxplot_out_folder.joinpath(f"{specie}")

    sent_boxplots(species_df, boxplot_out_path, species_df.columns[3:43], 6, 7)

##### Sentinel 2

In [ ]:
boxplot_out_folder = out_folder.joinpath(f"intra/sentinel_2")

if not boxplot_out_folder.exists():
    boxplot_out_folder.mkdir(parents=True)

for specie in range(0,14,1):
    species_df = merged_df_wide[merged_df_wide['species']==specie]

    boxplot_out_path = out_folder.joinpath(f"intra/sentinel_2/{specie}")

    sent_boxplots(species_df, boxplot_out_path, species_df.columns[43:89], 8, 6)

## Inter Species

#### Sentinel 1

In [ ]:
boxplot_out_folder = out_folder.joinpath(f"inter/sentinel_1")

if not boxplot_out_folder.exists():
    boxplot_out_folder.mkdir(parents=True)

for target in west_low_species:
    for control in mid_line_species:

        species = [target, control]
        species_df = merged_df_wide[merged_df_wide['species'].isin(species)]

        boxplot_out_path = out_folder.joinpath(f"inter/sentinel_1/{target}_{control}")

        sent_boxplots(species_df, boxplot_out_path, species_df.columns[3:43], 6, 7)

#### Sentinel 2

In [ ]:
boxplot_out_folder = out_folder.joinpath(f"inter/sentinel_2")

if not boxplot_out_folder.exists():
    boxplot_out_folder.mkdir(parents=True)

for target in west_low_species:
    for control in mid_line_species:

        species = [target, control]
        species_df = merged_df_wide[merged_df_wide['species'].isin(species)]

        boxplot_out_path = out_folder.joinpath(f"inter/sentinel_2/{target}_{control}")

        sent_boxplots(species_df, boxplot_out_path, species_df.columns[43:89], 8, 6)

: 